This is our proposed model, similar to the one in BERTMODEL_AmazonDS.ipynb file, only difference is it is trained and tested with IMDB data.


This is performed to understand whether our proposed model gives same results with different kinds of input review texts(like imdb reviews)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np

In [ ]:
# Installing the transformers package for BERT
!pip install transformers

     |████████████████████████████████| 3.1 MB 8.9 MB/s 
     |████████████████████████████████| 61 kB 519 kB/s 
     |████████████████████████████████| 3.3 MB 44.4 MB/s 
     |████████████████████████████████| 596 kB 45.0 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
import re
import string
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Data cleaning for IMDB data
def classify(x):
    if x == "pos":
        return 2
    if x=="unsup":
        return 1 
    return 0

In [ ]:
def clean_dataframe(df):
  # creates new column with corresponding class labels, the output variable.
  df['y'] = df['label'].apply(classify)
  df['label'] = df['y']

  # dropping all NaN values from the column reviewText
  df = df.dropna(axis=0, subset=['review'])
  return df

In [ ]:
# removes all the punctuations for the strings of reviewText, ie '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

In [ ]:
def text_process(df):
  # Removing all the punctuations from the words, and changing the words to lower case to maintain uniformity
  df['review']=df['review'].apply(lambda x: remove_punctuation(x.lower()))
  # stemming
  stemmer = PorterStemmer()
  # stop words are the words like "the, I, our etc"
  words = stopwords.words("english") 
  df['cleaned_reviews'] = df['review'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
  return df

In [ ]:
def process_df(df):
  df = clean_dataframe(df)
  df = text_process(df)
  return df

In [ ]:
URL_UNCLEAN_IMDB = "/content/drive/MyDrive/imdb_master.csv"
URL_CLEAN_IMDB = "/content/drive/MyDrive/imdb_master_10Kcleaned.csv"

In [ ]:
df_unclean = pd.read_csv(URL_UNCLEAN_IMDB, encoding = "ISO-8859-1")
  #print("UNCLEANED DATASET HEAD:\n",df_unclean.head(),"\n")
print(df_unclean.shape)
df_unclean = df_unclean.sample(n = 10000)
  #df_unclean = df_unclean.iloc[0:10000,:]
print(df_unclean.shape)
df_clean = process_df(df_unclean)
print(df_clean.shape)
print("CLEANED DATASET HEAD:\n",df_clean.head(),"\n")

  #writing to dataframe
df_clean.to_csv(URL_CLEAN_IMDB)

# once the caleaned data is stored to a file, it can be directly loaded and use next time instead of cleaning everytime

(100000, 5)
(10000, 5)
(10000, 7)
CLEANED DATASET HEAD:
        Unnamed: 0   type  ...  y                                    cleaned_reviews
43068       43068  train  ...  2  alex winter keanu reev return two dope san dim...
81660       81660  train  ...  1  ive seen hand master horror episod far must sa...
97597       97597  train  ...  1  reason gave aw film instead wonder cinematogra...
65468       65468  train  ...  1  far im concern woodi allen best stori tell nyc...
77325       77325  train  ...  1  rocki bullwinkl movi never made complet wast a...

[5 rows x 7 columns] 



In [ ]:
# Splitting the dataset into a 85-15 train-test split
from sklearn.model_selection import train_test_split
X_train_imdb, X_val_imdb, y_train_imdb, y_val_imdb = train_test_split(df_clean.index.values, 
                                                  df_clean.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df_clean.label.values)
print(X_train_imdb)
print(X_val_imdb)
# creating a column data_type to denote whether each record is for test or train
df_clean['data_type'] = ['not_set']*df_clean.shape[0]
df_clean.loc[X_train_imdb, 'data_type'] = 'train'
df_clean.loc[X_val_imdb, 'data_type'] = 'val'
df_clean.groupby(['y', 'label', 'data_type']).count()

[24327 95488 96828 ...  8896  7000 88332]
[10928 29422 98041 ... 16343 41862 93654]


Unnamed: 0  type  review  file  cleaned_reviews
y label data_type                                                 
0 0     train            2061  2061    2061  2061             2061
        val               363   363     363   363              363
1 1     train            4253  4253    4253  4253             4253
        val               751   751     751   751              751
2 2     train            2186  2186    2186  2186             2186
        val               386   386     386   386              386

In [ ]:
# Bert tokenizer initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)  
# Encoding training batch                           
encoded_data_train2 = tokenizer.batch_encode_plus(
    df_clean[df_clean.data_type=='train'].review.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train_imdb = encoded_data_train2['input_ids']
attention_masks_train_imdb = encoded_data_train2['attention_mask']
labels_train_imdb = torch.tensor(df_clean[df_clean.data_type=='train'].label.values)

dataset_train = TensorDataset(input_ids_train_imdb, attention_masks_train_imdb, labels_train_imdb)

# Encoding validation batch                           
encoded_data_val2 = tokenizer.batch_encode_plus(
    df_clean[df_clean.data_type=='val'].review.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
input_ids_val_imdb = encoded_data_val2['input_ids']
attention_masks_val_imdb = encoded_data_val2['attention_mask']
labels_val_imdb = torch.tensor(df_clean[df_clean.data_type=='val'].label.values)

dataset_val = TensorDataset(input_ids_val_imdb, attention_masks_val_imdb, labels_val_imdb)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Import and initialization of dataloaders and batch size hyperparameter
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
# Creating a dictionary which contains all unique attributes in y, here these are the classes
possible_labels = df_clean.y.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = int(possible_label)
label_dict
# Changing column name from y to label for clarity
df_clean['label'] = [label_dict[str_label] for str_label in df_clean['y']]


In [ ]:
# Initializing the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Importing the adam optimizer and a scheduler to manage learning rate in an optimal fashion
from transformers import AdamW, get_linear_schedule_with_warmup

# Initializing the optimizer (uses the adam algorithm - an extension to stochastic gradient descent)
optimizer = AdamW(model.parameters(),
                  lr=3e-5, 
                  eps=1e-8)
                  
# Setting epoch hyperparameter
epochs = 2

# Scheduler to increase learning rate for a certain number of steps then linearly deacrese the same
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
# Defining functions for f1 score and accuracy
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    correct=0
    total=0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        correct+=len(y_preds[y_preds==label])
        total+=len(y_true)
    print(f'Bert Model Accuracy Overall :',correct/total)

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
import random

# Sets torchs seed to randomize to enable usage of torch.random()
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Evaluate function to be used 
# tqdm to display the live progress of the model training, calculates average loss, predictions and true values.
def evaluate(dataloader_val):

    # used to turn off layers that dont perform as expected
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
  

Training and validating the data using our BERT model selected above

In [ ]:
# Train the model
epochs =2
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
      
    # Write results  
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Evaluating the model predictions using sklearn metrics like confusion_matrix and classification reports

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
target_names = ['Negative', 'Neutral', 'positive']

pred_arr = np.argmax(predictions, 1)
conf_matrix_imdb=confusion_matrix(true_vals, pred_arr)
print("Confusion matrix for IMDB Data using BERT model")
print(conf_matrix_imdb)
print(classification_report(true_vals, pred_arr, target_names=target_names))